In [1]:
import pickle
import numpy as np

from sklearn.preprocessing import scale

#from keras.models import Sequential
#from keras.optimizers import SGD, Adam, Adagrad
#from keras import backend as K
#from keras.layers import Embedding
#from keras.layers import Dense, Reshape, Concatenate, Activation, Dropout
#from keras.callbacks import ModelCheckpoint
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In [2]:
train_cache = 'cache/train2.pickle'
train_labels_cache = 'cache/train-labels2.npy'
validation_cache = 'cache/validation2.pickle'
validation_labels_cache = 'cache/validation-labels2.npy'
test_cache = 'cache/test2.pickle'
test_labels_cache = 'cache/test-labels2.npy'
competition_test_cache = 'cache/competition-test2.pickle'
metadata_cache = 'cache/metadata2.pickle'

In [6]:
train_cache = 'cache/train.pickle'
train_labels_cache = 'cache/train-labels.npy'
validation_cache = 'cache/validation.pickle'
validation_labels_cache = 'cache/validation-labels.npy'
test_cache = 'cache/test.pickle'
test_labels_cache = 'cache/test-labels.npy'
competition_test_cache = 'cache/competition-test.pickle'
metadata_cache = 'cache/metadata.pickle'

In [7]:
train = pd.read_pickle(train_cache)
validation = pd.read_pickle(validation_cache)
test = pd.read_pickle(test_cache)

train_labels = np.load(train_labels_cache)
validation_labels = np.load(validation_labels_cache)
test_labels = np.load(test_labels_cache)

competition_test = pd.read_pickle(competition_test_cache)
with open(metadata_cache, 'rb') as handle:
    metadata = pickle.load(handle)

In [11]:
def process_features(df):
    return np.array(df[['QUARTER_HOUR','DAY_OF_WEEK','WEEK_OF_YEAR','TAXI_ID_ENCODED','STAND_LONGITUDE','STAND_LATITUDE']].values)

In [12]:
reg1 = GradientBoostingRegressor(random_state=0)

In [13]:
reg1.fit(process_features(train),train_labels)

GradientBoostingRegressor(random_state=0)

In [14]:
reg1.predict(process_features(competition_test))

array([ 795.56138425,  765.66444183,  795.53669363,  721.15813314,
        764.48464917,  780.70289996,  799.26166203,  780.70289996,
        818.44179814,  795.53669363,  726.52588534,  776.40397012,
        775.29084867,  816.38878714,  778.23065784,  779.13217405,
        777.51686229,  779.13217405,  795.04509687,  778.23065784,
        793.92138186,  776.40397012,  777.51686229,  787.14126824,
        857.704712  ,  808.00963997,  778.94101574,  794.63517741,
        729.51632681,  804.21777398,  778.94101574,  767.91263298,
        779.13217405,  787.86049684,  779.13217405,  779.13217405,
        776.40397012,  740.54050226,  790.06447128,  777.06674601,
        729.94114734,  779.13217405,  779.15686467,  819.34331435,
        761.84529709,  821.49756121,  787.81325441,  776.40397012,
        682.76063544,  721.11383405,  797.53118427,  753.062908  ,
        738.99513677,  795.56138425,  792.80848969,  735.98640225,
        819.36800497,  739.88539857,  780.70289996,  775.29084

In [16]:
import math
from sklearn.metrics import mean_squared_error
validation_loss = math.sqrt(mean_squared_error(reg1.predict(process_features(validation)),validation_labels))
print(validation_loss)
test_loss = math.sqrt(mean_squared_error(reg1.predict(process_features(test)),test_labels))
print(test_loss)

484.9021940803681
502.7348237696494


In [18]:
xgb_predict = pd.read_csv('test_public.csv')
xgb_predict = xgb_predict['TRIP_ID']
predict_array = reg1.predict(process_features(competition_test))
xgb_predict= pd.concat([xgb_predict, pd.DataFrame(predict_array)], axis=1)
xgb_predict = xgb_predict.rename(columns={0: 'TRAVEL_TIME'})
xgb_predict.head()

,TRIP_ID,TRAVEL_TIME
0,T1,795.561384
1,T2,765.664442
2,T3,795.536694
3,T4,721.158133
4,T5,764.484649


In [19]:
xgb_predict['TRAVEL_TIME'].mean()

721.6838573189444

In [20]:
xgb_predict.to_csv('xgb_untruncated_approach1.csv', index=False)

In [4]:
reg2 = GradientBoostingRegressor(random_state=0)

In [ ]:
df = train[['QUARTER_HOUR','DAY_OF_WEEK','WEEK_OF_YEAR','ORIGIN_STAND_ENCODED'
                           'dt_2014-08-14 00:00:00','dt_2014-09-30 00:00:00','dt_2014-10-06 00:00:00','dt_2014-11-01 00:00:00',
                           'dt_2014-12-21 00:00:00']]
comp = competition_test[['QUARTER_HOUR','DAY_OF_WEEK','WEEK_OF_YEAR','ORIGIN_STAND_ENCODED'
                           'dt_2014-08-14 00:00:00','dt_2014-09-30 00:00:00','dt_2014-10-06 00:00:00','dt_2014-11-01 00:00:00',
                           'dt_2014-12-21 00:00:00']]
df.head()
df.shape

In [ ]:
merged = pd.concat([df,comp],axis=0)
merged.head()
merged.shape

In [ ]:
merged = pd.get_dummies(merged, columns = ['ORIGIN_STAND_ENCODED'])

In [ ]:
df = merged.head(-320)
comp = merged.tail(320)

In [ ]:
df.head()

In [ ]:
comp.head()

In [ ]:
reg2.fit(np.array(df.values),train_labels)

In [ ]:
reg2.predict(np.array(comp.values))

In [ ]:
xgb_predict = pd.read_csv('test_public.csv')
xgb_predict = xgb_predict['TRIP_ID']
predict_array = reg2.predict(np.array(comp.values))
xgb_predict= pd.concat([xgb_predict, pd.DataFrame(predict_array)], axis=1)
xgb_predict = xgb_predict.rename(columns={0: 'TRAVEL_TIME'})
xgb_predict.head()

In [ ]:
xgb_predict['TRAVEL_TIME'].mean()

In [ ]:
xgb_predict.to_csv('xgb_greater300_truncated.csv', index=False)